# Sťahovanie dát ribbons z ribbondb_v1.0.csv

Sťahovanie ribbons dát z datasetu ribbondb_v1.0.csv 5 pred + PEAK + 5 po

Obrázky uložené v priečinku "ribbons"

inštalácia knižníc

In [11]:
pip install h5py==3.7.0 astropy==6.1.4 parfive==1.5.1 Glymur==0.13.6 sunpy==3.1.0 numpy==1.24.4 hvpy==1.1.0 aiapy==0.7.4 opencv-python==4.10.0.84 ultralytics torch

  Using cached h5py-3.7.0.tar.gz (392 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached astropy-6.1.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (10 kB)
  Using cached Glymur-0.13.6-py3-none-any.whl.metadata (1.0 kB)
  Using cached numpy-1.24.4.tar.gz (10.9 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [33 lines of output]
      Traceback (most recent call last):
        File "/opt/conda/lib/python3.12/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 389, in <module>
          main()
        File "/opt/conda/lib/python3.12/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 373, in main
          json_out["return_val"] = hook(**hook_in

In [4]:
pip install --upgrade pip setuptools #3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 19.0 MB/s eta 0:00:0000:010:01
  Using cached setuptools-75.5.0-py3-none-any.whl (1.2 MB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 65.3.0
    Uninstalling setuptools-65.3.0:
      Successfully uninstalled setuptools-65.3.0
  Attempting uninstall: pip
    Found existing installation: pip 22.2.2
    Uninstalling pip-22.2.2:
      Successfully uninstalled pip-22.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.55.2 requires numpy<1.23,>=1.18, but you have numpy 2.1.3 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install backports.tarfile #4

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install packaging==21.3 #2

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import os
from sunpy.map import Map
import hvpy
import cv2
from datetime import datetime, timedelta
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.io import fits

#from aiapy.calibrate import normalize_exposure

Načítanie datasetu

In [7]:
data = pd.read_csv("csv's/ribbondb_v1.0.csv")

In [8]:
data

,INDEX,KEY,TSTART [UT],TPEAK [UT],TFINAL [UT],IXPEAK [W/m^2],LAT [deg],LON [deg],ARNUM,PHI_AR [Mx],PHI_RBN [Mx],DPHI_RBN [Mx],S_AR [cm^2],S_RBN [cm^2],DS_RBN [cm^2],R_PHI [%],R_S[%]
0,0,20100612_0030_11081_M2.0,2010-06-12 00:30,2010-06-12 00:57,2010-06-12 01:02,0.000020,23,43,11081,1.839000e+22,3.209000e+21,7.712000e+20,9.199000e+19,4.441000e+18,1.158000e+18,17.5,4.8
1,1,20100713_1043_11087_C2.6,2010-07-13 10:43,2010-07-13 10:51,2010-07-13 10:55,0.000003,19,-26,11087,2.876000e+22,7.220000e+20,1.040000e+20,8.642000e+19,1.385000e+18,2.332000e+17,2.5,1.6
2,2,20100727_0412_11089_C2.2,2010-07-27 04:12,2010-07-27 04:24,2010-07-27 04:34,0.000002,-24,21,11089,3.280000e+22,1.617000e+21,4.311000e+20,1.043000e+20,3.394000e+18,9.310000e+17,4.9,3.3
3,3,20100801_0755_11092_C3.2,2010-08-01 07:55,2010-08-01 08:26,2010-08-01 09:35,0.000003,20,-36,11092,2.855000e+22,2.956000e+21,1.161000e+21,8.681000e+19,4.721000e+18,2.247000e+18,10.4,5.4
4,4,20100807_1755_11093_M1.0,2010-08-07 17:55,2010-08-07 18:24,2010-08-07 18:47,0.000010,11,-34,11093,3.031000e+22,4.754000e+21,5.771000e+20,8.570000e+19,9.502000e+18,1.229000e+18,15.7,11.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3132,3132,20160414_1321_12529_C1.8,2016-04-14 13:21,2016-04-14 13:32,2016-04-14 13:44,0.000002,11,14,12529,3.775000e+22,1.229000e+21,2.494000e+20,7.646000e+19,2.117000e+18,5.865000e+17,3.3,2.8
3133,3133,20160414_1759_12529_C1.3,2016-04-14 17:59,2016-04-14 18:07,2016-04-14 18:16,0.000001,12,7,12529,4.289000e+22,7.151000e+20,2.401000e+20,9.266000e+19,1.253000e+18,4.997000e+17,1.7,1.4
3134,3134,20160415_1404_12529_C4.1,2016-04-15 14:04,2016-04-15 14:30,2016-04-15 14:43,0.000004,11,20,12529,4.702000e+22,1.305000e+21,4.510000e+20,9.426000e+19,2.840000e+18,1.113000e+18,2.8,3.0
3135,3135,20160416_1942_12529_C5.8,2016-04-16 19:42,2016-04-16 19:58,2016-04-16 20:12,0.000006,13,43,12529,4.765000e+22,1.532000e+21,3.555000e+20,8.499000e+19,2.730000e+18,6.568000e+17,3.2,3.2


In [9]:
data.columns

Index(['INDEX', ' KEY', ' TSTART [UT]', ' TPEAK [UT]', ' TFINAL [UT]',
       ' IXPEAK [W/m^2]', ' LAT [deg]', ' LON [deg]', ' ARNUM', ' PHI_AR [Mx]',
       ' PHI_RBN [Mx]', ' DPHI_RBN [Mx]', ' S_AR [cm^2]', ' S_RBN [cm^2]',
       ' DS_RBN [cm^2]', ' R_PHI [%]', ' R_S[%]'],
      dtype='object')

In [10]:
data.head()

,INDEX,KEY,TSTART [UT],TPEAK [UT],TFINAL [UT],IXPEAK [W/m^2],LAT [deg],LON [deg],ARNUM,PHI_AR [Mx],PHI_RBN [Mx],DPHI_RBN [Mx],S_AR [cm^2],S_RBN [cm^2],DS_RBN [cm^2],R_PHI [%],R_S[%]
0,0,20100612_0030_11081_M2.0,2010-06-12 00:30,2010-06-12 00:57,2010-06-12 01:02,0.000020,23,43,11081,1.839000e+22,3.209000e+21,7.712000e+20,9.199000e+19,4.441000e+18,1.158000e+18,17.5,4.8
1,1,20100713_1043_11087_C2.6,2010-07-13 10:43,2010-07-13 10:51,2010-07-13 10:55,0.000003,19,-26,11087,2.876000e+22,7.220000e+20,1.040000e+20,8.642000e+19,1.385000e+18,2.332000e+17,2.5,1.6
2,2,20100727_0412_11089_C2.2,2010-07-27 04:12,2010-07-27 04:24,2010-07-27 04:34,0.000002,-24,21,11089,3.280000e+22,1.617000e+21,4.311000e+20,1.043000e+20,3.394000e+18,9.310000e+17,4.9,3.3
3,3,20100801_0755_11092_C3.2,2010-08-01 07:55,2010-08-01 08:26,2010-08-01 09:35,0.000003,20,-36,11092,2.855000e+22,2.956000e+21,1.161000e+21,8.681000e+19,4.721000e+18,2.247000e+18,10.4,5.4
4,4,20100807_1755_11093_M1.0,2010-08-07 17:55,2010-08-07 18:24,2010-08-07 18:47,0.000010,11,-34,11093,3.031000e+22,4.754000e+21,5.771000e+20,8.570000e+19,9.502000e+18,1.229000e+18,15.7,11.1


In [15]:
data['className'] = data[' KEY'].str.extract(r'_([A-Z]\d)\.\d')
classCounts2 = data['className'].value_counts()
classCounts2 = classCounts2.sort_index() 

In [16]:
classCounts2

className
C1    1271
C2     641
C3     317
C4     198
C5     138
C6     100
C7      63
C8      57
C9      46
M1     170
M2      48
M3      22
M4      14
M5       9
M6      12
M7       5
M8       4
M9       3
X1      13
X2       4
X3       1
X5       1
Name: count, dtype: int64

In [16]:
# nastavenie priecinkov
base_folder = "./ribbons"
segmented_folder = "./ribbons/segmented"
class_folders = {'C': 'C', 'M': 'M', 'X': 'X'}
for cls in class_folders.values():
    os.makedirs(os.path.join(base_folder, cls), exist_ok=True)
    os.makedirs(os.path.join(segmented_folder, cls), exist_ok=True)

# Sťahovanie obrázkov z datasetu ribbondb_v1.0.csv do priečinku ribbons -> segmented

fits + jp2 výrezy

In [53]:
# ziskanie source ID pre SDO AIA 1600
data_sources = hvpy.getDataSources()
source_id = data_sources['SDO']['AIA']['1600']['sourceId']

# iteracia po jednotlivych riadkoch
for index, row in data.iterrows():
    key = row[' KEY']
    event_class = key[-4]  # vyber triedy podla stlpca KEY

    if event_class not in class_folders:
        continue  
    
    tpeak = datetime.strptime(row[' TPEAK [UT]'].strip(), '%Y-%m-%d %H:%M')
    tstart = datetime.strptime(row[' TSTART [UT]'].strip(), '%Y-%m-%d %H:%M')
    tfinal = datetime.strptime(row[' TFINAL [UT]'].strip(), '%Y-%m-%d %H:%M')
    duration = (tfinal - tstart).total_seconds() / 60

    # casy: chceme obrazky udalosti v case PEAK, 5 obr v rozmedzi 1 minuty pred a 5 po PEAK
    if duration >= 11:
        times = [tpeak - timedelta(minutes=5) + timedelta(minutes=i) for i in range(11)]
    else:
        times = pd.date_range(tstart, tfinal, freq='min').to_pydatetime().tolist()
    
    img_lat = row[' LAT [deg]']
    img_lon = row[' LON [deg]']
    
    # suradnice pre vyrez
    top_right = SkyCoord((img_lon - 20) * u.deg, (img_lat + 20) * u.deg, frame='heliographic_stonyhurst')
    bottom_left = SkyCoord((img_lon + 20) * u.deg, (img_lat - 20) * u.deg, frame='heliographic_stonyhurst')

    # priecinky
    class_folder = os.path.join(base_folder, class_folders[event_class])
    cutout_class_folder = os.path.join(segmented_folder, class_folders[event_class])

    # vytvorenie priecinkov, ak neexistuju
    os.makedirs(class_folder, exist_ok=True)
    os.makedirs(cutout_class_folder, exist_ok=True)

    # stahovanie obrazku pre dane casy
    for time in times:
        date_str = time.strftime('%Y-%m-%dT%H:%M')
        is_peak = time == tpeak
        level = key[-3:]
        filename_suffix = f"{time.strftime('%Y%m%d_%H%M')}_{event_class}{level}"
        filename = f"{filename_suffix}_PEAK.jp2" if is_peak else f"{filename_suffix}.jp2"
        
        jp2_path = os.path.join(class_folder, filename)

        # stahovanie JP2 obrazku
        print(f"Attempting to download image for date: {date_str} with source ID: {source_id}")
        try:
            image_content = hvpy.getJP2Image(date=date_str, sourceId=source_id)
            if image_content:
                with open(jp2_path, 'wb') as f:
                    f.write(image_content)
                print(f"Image saved to {jp2_path}")
            else:
                print(f"No content retrieved for {date_str}. Skipping.")
                continue
        except Exception as e:
            print(f"Error retrieving image for {date_str}: {e}")
            continue
        
        
        if not os.path.exists(jp2_path):
            print(f"JP2 file does not exist at {jp2_path}. Skipping.")
            continue

        file = Map(jp2_path)
        # try:
        #     file = normalize_exposure(file)
        # except Exception as e:
        #     print(f"Error normalizing exposure for {jp2_path}: {e}")
        #     continue

        # výrez
        aia_sub = file.submap(bottom_left, top_right=top_right)
        cutout_filename = f"{filename_suffix}_cutout.fits"
        cutout_path = os.path.join(cutout_class_folder, cutout_filename)
        
        try:
            aia_sub.save(cutout_path, overwrite=True)
            # konverzai na JP2
            with fits.open(cutout_path) as hdul:
                fits_data = hdul[0].data
                cv2.imwrite(cutout_path.replace('.fits', '.jp2'), fits_data)
            os.remove(jp2_path)
            print(f"Cutout saved to {cutout_path}")
        except Exception as e:
            print(f"Error saving cutout {cutout_path}: {e}")
            continue


Attempting to download image for date: 2010-06-12T00:52 with source ID: 15
Image saved to ./experiment1/M/20100612_0052_M2.0.jp2


/opt/conda/lib/python3.10/site-packages/sunpy/io/fits.py:228: SunpyMetadataWarning: The meta key oscnmean has a NaN value, which is not valid in a FITS header, dropping from the FITS header
  warn_metadata(f'The meta key {k} has a NaN value, which is not valid in a FITS '
/opt/conda/lib/python3.10/site-packages/sunpy/io/fits.py:228: SunpyMetadataWarning: The meta key oscnrms has a NaN value, which is not valid in a FITS header, dropping from the FITS header
  warn_metadata(f'The meta key {k} has a NaN value, which is not valid in a FITS '
/opt/conda/lib/python3.10/site-packages/sunpy/io/fits.py:228: SunpyMetadataWarning: The meta key rsun_lf has a NaN value, which is not valid in a FITS header, dropping from the FITS header
  warn_metadata(f'The meta key {k} has a NaN value, which is not valid in a FITS '
/opt/conda/lib/python3.10/site-packages/sunpy/io/fits.py:228: SunpyMetadataWarning: The meta key x0_lf has a NaN value, which is not valid in a FITS header, dropping from the FITS hea

Cutout saved to ./experiment1/segmented/M/20100612_0052_M2.0_cutout.fits
Attempting to download image for date: 2010-06-12T00:53 with source ID: 15
Image saved to ./experiment1/M/20100612_0053_M2.0.jp2


/opt/conda/lib/python3.10/site-packages/sunpy/io/fits.py:228: SunpyMetadataWarning: The meta key oscnmean has a NaN value, which is not valid in a FITS header, dropping from the FITS header
  warn_metadata(f'The meta key {k} has a NaN value, which is not valid in a FITS '
/opt/conda/lib/python3.10/site-packages/sunpy/io/fits.py:228: SunpyMetadataWarning: The meta key oscnrms has a NaN value, which is not valid in a FITS header, dropping from the FITS header
  warn_metadata(f'The meta key {k} has a NaN value, which is not valid in a FITS '
/opt/conda/lib/python3.10/site-packages/sunpy/io/fits.py:228: SunpyMetadataWarning: The meta key rsun_lf has a NaN value, which is not valid in a FITS header, dropping from the FITS header
  warn_metadata(f'The meta key {k} has a NaN value, which is not valid in a FITS '
/opt/conda/lib/python3.10/site-packages/sunpy/io/fits.py:228: SunpyMetadataWarning: The meta key x0_lf has a NaN value, which is not valid in a FITS header, dropping from the FITS hea

Cutout saved to ./experiment1/segmented/M/20100612_0053_M2.0_cutout.fits
Attempting to download image for date: 2010-06-12T00:54 with source ID: 15
Image saved to ./experiment1/M/20100612_0054_M2.0.jp2


...

/opt/conda/lib/python3.10/site-packages/sunpy/io/fits.py:228: SunpyMetadataWarning: The meta key oscnmean has a NaN value, which is not valid in a FITS header, dropping from the FITS header
  warn_metadata(f'The meta key {k} has a NaN value, which is not valid in a FITS '
/opt/conda/lib/python3.10/site-packages/sunpy/io/fits.py:228: SunpyMetadataWarning: The meta key oscnrms has a NaN value, which is not valid in a FITS header, dropping from the FITS header
  warn_metadata(f'The meta key {k} has a NaN value, which is not valid in a FITS '
/opt/conda/lib/python3.10/site-packages/sunpy/io/fits.py:228: SunpyMetadataWarning: The meta key rsun_lf has a NaN value, which is not valid in a FITS header, dropping from the FITS header
  warn_metadata(f'The meta key {k} has a NaN value, which is not valid in a FITS '
/opt/conda/lib/python3.10/site-packages/sunpy/io/fits.py:228: SunpyMetadataWarning: The meta key x0_lf has a NaN value, which is not valid in a FITS header, dropping from the FITS hea

Cutout saved to ./experiment1/segmented/M/20100612_0100_M2.0_cutout.fits
Attempting to download image for date: 2010-06-12T01:01 with source ID: 15
Image saved to ./experiment1/M/20100612_0101_M2.0.jp2


/opt/conda/lib/python3.10/site-packages/sunpy/io/fits.py:228: SunpyMetadataWarning: The meta key oscnmean has a NaN value, which is not valid in a FITS header, dropping from the FITS header
  warn_metadata(f'The meta key {k} has a NaN value, which is not valid in a FITS '
/opt/conda/lib/python3.10/site-packages/sunpy/io/fits.py:228: SunpyMetadataWarning: The meta key oscnrms has a NaN value, which is not valid in a FITS header, dropping from the FITS header
  warn_metadata(f'The meta key {k} has a NaN value, which is not valid in a FITS '
/opt/conda/lib/python3.10/site-packages/sunpy/io/fits.py:228: SunpyMetadataWarning: The meta key rsun_lf has a NaN value, which is not valid in a FITS header, dropping from the FITS header
  warn_metadata(f'The meta key {k} has a NaN value, which is not valid in a FITS '
/opt/conda/lib/python3.10/site-packages/sunpy/io/fits.py:228: SunpyMetadataWarning: The meta key x0_lf has a NaN value, which is not valid in a FITS header, dropping from the FITS hea

Cutout saved to ./experiment1/segmented/M/20100612_0101_M2.0_cutout.fits
Attempting to download image for date: 2010-06-12T01:02 with source ID: 15
Image saved to ./experiment1/M/20100612_0102_M2.0.jp2
Cutout saved to ./experiment1/segmented/M/20100612_0102_M2.0_cutout.fits


/opt/conda/lib/python3.10/site-packages/sunpy/io/fits.py:228: SunpyMetadataWarning: The meta key oscnmean has a NaN value, which is not valid in a FITS header, dropping from the FITS header
  warn_metadata(f'The meta key {k} has a NaN value, which is not valid in a FITS '
/opt/conda/lib/python3.10/site-packages/sunpy/io/fits.py:228: SunpyMetadataWarning: The meta key oscnrms has a NaN value, which is not valid in a FITS header, dropping from the FITS header
  warn_metadata(f'The meta key {k} has a NaN value, which is not valid in a FITS '
/opt/conda/lib/python3.10/site-packages/sunpy/io/fits.py:228: SunpyMetadataWarning: The meta key rsun_lf has a NaN value, which is not valid in a FITS header, dropping from the FITS header
  warn_metadata(f'The meta key {k} has a NaN value, which is not valid in a FITS '
/opt/conda/lib/python3.10/site-packages/sunpy/io/fits.py:228: SunpyMetadataWarning: The meta key x0_lf has a NaN value, which is not valid in a FITS header, dropping from the FITS hea